In traditional robotic systems, joint torque sensors are used to measure the total torque at a joint, including both internal and external components. However, due to the inherent properties of HDTs, such as high friction and complex mechanical behavior, a significant portion of the measured torque does not contribute to the actual movement of the robot's link but is rather dissipated internally.

To address this, the current research paradigm emphasizes the importance of isolating and accurately estimating the external torque component. External torques are critical as they directly affect the robot's interaction with its environment, such as during tasks involving physical manipulation or contact with objects. While the internal torques due to friction and position-dependent effects are known and can be compensated for, the external torque estimation remains challenging and is the focus of this work.

This thesis explores various methods to estimate these external torques, leveraging both traditional analytical approaches and modern machine learning techniques. The analytical methods provide a foundation by modeling known internal forces, while the machine learning approaches aim to capture the residuals and complexities not accounted for by these models, particularly the external torques. This dual approach not only enhances the accuracy of torque estimation but also contributes to the development of a more comprehensive virtual joint torque sensor.

#### Introduce motor currents as a method of estimating joint torque

### State of the Art

The estimation of interaction forces and torques in robotic systems, particularly in robotic joints utilizing harmonic drive transmissions, has been extensively researched due to its critical importance in ensuring precise control and safe operation, especially in dynamic environments and human-robot interaction (HRI). Over the past decades, various methodologies have been proposed, ranging from sensor-based techniques to sensorless approaches and hybrid systems. Each method offers unique benefits while also presenting specific challenges that limit its applicability in certain scenarios. This chapter provides an in-depth review of the state-of-the-art techniques, incorporating technical details, equations, and a critical analysis of their respective advantages and limitations.

#### 1. **Sensor-Based Approaches**

Force/torque (F/T) sensors are among the most direct and reliable methods for measuring interaction forces and moments in robotic systems. These sensors are typically mounted on the robot's end-effector and provide real-time measurements of forces and torques at the point of contact. The force \(\mathbf{F}\) and torque \(\mathbf{T}\) measured by such sensors can be represented as:

$$
\mathbf{F}_{meas} = \mathbf{F}_{ext} + \mathbf{F}_{noise}
$$

$$
\mathbf{T}_{meas} = \mathbf{T}_{ext} + \mathbf{T}_{noise}
$$

where \(\mathbf{F}_{ext}\) and \(\mathbf{T}_{ext}\) are the true external force and torque vectors, and \(\mathbf{F}_{noise}\) and \(\mathbf{T}_{noise}\) represent sensor noise, which is typically assumed to be Gaussian.

**Buondonno and De Luca (2016)** explored a hybrid approach that combined these physical sensors with virtual sensors based on model-based estimations. In their approach, a force/torque sensor was placed at the robot base, allowing for the measurement of the reaction forces and moments that arise from external interactions anywhere along the robot's structure. The base force/torque sensor can be used in conjunction with the recursive Newton-Euler algorithm to compute the total force and moment acting on each link:

$$
\mathbf{F}_i = m_i \mathbf{a}_{ci} 
$$

$$
\mathbf{T}_i = \mathbf{I}_i \dot{\boldsymbol{\omega}}_i + \boldsymbol{\omega}_i \times (\mathbf{I}_i \boldsymbol{\omega}_i)
$$

where \(m_i\) is the mass of link \(i\), \(\mathbf{a}_{ci}\) is the acceleration of the center of mass, \(\mathbf{I}_i\) is the inertia tensor, and \(\boldsymbol{\omega}_i\) is the angular velocity of the link.

The hybrid approach is robust in detecting and measuring forces acting on any part of the robot, but it still faces significant challenges:
- **Localized Sensing**: While the base-mounted sensor can detect forces along the structure, it may not precisely identify the location of the force application, particularly when multiple forces act simultaneously on different links.
- **Cost and Integration**: The integration of physical sensors, especially in complex robotic systems, increases the overall cost and complexity. Additionally, such sensors are susceptible to damage or failure in harsh environments, limiting their practical use in certain applications.

These challenges have driven the development of alternative sensorless methods that can estimate forces and torques without relying on physical sensors.

#### 2. **Sensorless Approaches**

Sensorless estimation methods utilize the robot’s internal signals—such as motor currents, joint positions, and velocities—to infer the forces and torques acting on the system. These methods eliminate the need for expensive and fragile physical sensors, offering a cost-effective and robust alternative. A common sensorless approach involves using the relationship between motor torque \(\tau_m\), motor current \(I_m\), and external torque \(\tau_{ext}\), often expressed as:

$$
\tau_m = k_t I_m - \tau_{ext}
$$

where \(k_t\) is the motor’s torque constant.

**Ismail et al. (2021)** proposed a simplified sensorless torque estimation method tailored for harmonic drives, which are widely used in robotic joints due to their high reduction ratios and precision. Their method leverages the structural damping friction inherent in harmonic drives to estimate torque without relying on complex nonlinear models. The harmonic drive's compliance can be modeled as:

$$
\tau_{HD} = K_{HD} (\theta_m - \theta_l)
$$

where \(\tau_{HD}\) is the torque transmitted through the harmonic drive, \(K_{HD}\) is the drive’s stiffness, and \(\theta_m\) and \(\theta_l\) are the angular positions of the motor and load, respectively.

In their approach, the authors simplified the estimation by assuming that the structural damping coefficient \(B_{st}\) dominates the dynamic response, leading to a linear relationship between the torsional rate \(\dot{\theta}_{m,l}\) and the torque:

$$
\tau_{HD} \approx B_{st} \dot{\theta}_{m,l}
$$

This linear approximation reduces the computational burden and makes real-time torque estimation more feasible.

However, sensorless methods are not without their limitations:
- **Dependency on Accurate Dynamic Models**: The accuracy of sensorless torque estimation depends heavily on the precision of the dynamic model. Inaccuracies in model parameters, such as stiffness or damping coefficients, can lead to significant errors in torque estimation.
- **Sensitivity to Operating Conditions**: Sensorless methods are particularly sensitive to variations in operating conditions, such as changes in load or speed. Low-speed operations or environments with significant model uncertainties can degrade the accuracy of the estimation.

Despite these challenges, sensorless methods remain attractive due to their simplicity and cost-effectiveness, provided that the dynamic models are accurately maintained.

#### 3. **Kalman Filter-Based Estimation**

Kalman filters are widely used in robotics for their ability to provide optimal estimates of system states in the presence of noise and uncertainties. The Kalman filter estimates the system’s state vector \(\mathbf{x}\) (which may include joint positions, velocities, and forces) by recursively updating predictions based on new measurements. The state update equations in a Kalman filter are given by:

$$
\hat{\mathbf{x}}_k = \mathbf{A} \hat{\mathbf{x}}_{k-1} + \mathbf{B} \mathbf{u}_k + \mathbf{w}_k
$$

$$
\hat{\mathbf{P}}_k = \mathbf{A} \hat{\mathbf{P}}_{k-1} \mathbf{A}^T + \mathbf{Q}
$$

where \(\hat{\mathbf{x}}_k\) is the estimated state at time step \(k\), \(\mathbf{A}\) is the state transition matrix, \(\mathbf{B}\) is the control input matrix, \(\mathbf{u}_k\) is the control input, \(\mathbf{w}_k\) is the process noise, and \(\mathbf{Q}\) is the process noise covariance.

When new measurements \(\mathbf{z}_k\) are available, the Kalman filter updates the state estimate using the measurement update equations:

$$
\mathbf{K}_k = \hat{\mathbf{P}}_k \mathbf{H}^T (\mathbf{H} \hat{\mathbf{P}}_k \mathbf{H}^T + \mathbf{R})^{-1}
$$

$$
\hat{\mathbf{x}}_k = \hat{\mathbf{x}}_k + \mathbf{K}_k (\mathbf{z}_k - \mathbf{H} \hat{\mathbf{x}}_k)
$$

$$
\hat{\mathbf{P}}_k = (\mathbf{I} - \mathbf{K}_k \mathbf{H}) \hat{\mathbf{P}}_k
$$

where \(\mathbf{K}_k\) is the Kalman gain, \(\mathbf{H}\) is the measurement matrix, \(\mathbf{R}\) is the measurement noise covariance, and \(\mathbf{I}\) is the identity matrix.

**Wahrburg et al. (2017)** applied a Kalman filter to estimate Cartesian contact forces and torques in robotic manipulators using motor currents, joint angles, and joint speeds. This approach is particularly advantageous in scenarios where external sensors are not available or desirable. The Kalman filter can accurately estimate the external forces \(\mathbf{F}_{ext}\) and torques \(\mathbf{T}_{ext}\) acting on the robot’s end-effector by treating the joint torques as indirect measurements of these forces:

$$
\mathbf{F}_{ext} = \mathbf{J}^T \tau_{ext}
$$

where \(\mathbf{J}\) is the Jacobian matrix relating joint velocities to end-effector velocities.

The Kalman filter's recursive nature allows it to dynamically update the estimates of \(\mathbf{F}_{ext}\) and \(\mathbf{T}_{ext}\) as new measurements are obtained, making it robust to noise and model inaccuracies.

However, Kalman filter-based methods also face certain limitations:
- **Complexity and Calibration**: Implementing a Kalman filter requires careful calibration of the noise covariance matrices \(\mathbf{Q}\) and \(\mathbf{R}\), which significantly affects the filter’s performance. Improper tuning can lead to suboptimal estimates or slow convergence.
- **Model Dependency**: The performance of the Kalman filter is highly dependent on the accuracy of the system model. Inaccuracies in the robot’s dynamic model, particularly in the estimation of inertial properties or friction, can degrade the quality of the force and torque estimates.
- **Performance Near Singularities**: The Kalman filter's performance can deteriorate when the robot operates near kinematic singularities, where the Jacobian matrix \(\mathbf{J}\) becomes ill-conditioned, leading to poor force and torque estimation.

Despite these challenges, the Kalman filter remains a powerful tool for force and torque estimation, especially in situations where direct measurements are noisy or unavailable.

#### 4. **Hybrid Approaches: Combining Real and Virtual Sensors**

Hybrid approaches that combine real and virtual sensors represent an attempt to harness the strengths of both sensor-based and sensorless methods while mitigating their respective weaknesses. **Buondonno and De Luca (2016)** proposed such a hybrid approach, where a base-mounted force/torque sensor is combined with a virtual sensor based on model-based estimations. This virtual sensor is realized through the residual vector commonly used for collision detection:

$$
\mathbf{r}(t) = \mathbf{K} \left(\mathbf{m} - \int_0^t (\tau + \mathbf{C}(\mathbf{q}, \dot{\mathbf{q}}) \dot{\mathbf{q}} - \mathbf{g}(\mathbf{q})) dt' \right)
$$

where \(\mathbf{m} = \mathbf{B}(\mathbf{q}) \dot{\mathbf{q}}\) is the robot's generalized momentum, \(\mathbf{C}(\mathbf{q}, \dot{\mathbf{q}})\) is the Coriolis and centrifugal forces matrix, \(\mathbf{g}(\mathbf{q})\) is the gravity vector, and \(\mathbf{K}\) is a diagonal gain matrix.

This residual vector \(\mathbf{r}(t)\) is used to estimate the external torque \(\tau_{ext}\) as:

$$
\dot{\mathbf{r}}(t) = \mathbf{K}(\tau_{ext} - \mathbf{r}(t))
$$

By integrating this virtual sensor with the base force/torque sensor, the approach provides a more comprehensive estimation of the interaction forces and moments along the robot’s structure.

However, hybrid methods also have their limitations:
- **Complexity of Integration**: Combining real and virtual sensors increases the complexity of the system, both in terms of hardware integration and computational requirements. The fusion of data from multiple sources requires sophisticated algorithms to ensure consistency and reliability.
- **Cost and Maintenance**: While hybrid approaches aim to reduce reliance on physical sensors, the inclusion of even a few such sensors still adds to the overall cost and maintenance burden of the system.
- **Model Dependency**: Hybrid methods still rely on accurate dynamic models for the virtual sensors, meaning they inherit many of the challenges associated with model-based sensorless methods, such as calibration and sensitivity to parameter uncertainties.

These hybrid approaches represent a promising direction in the field of force and torque estimation, as they aim to combine the best aspects of both sensor-based and sensorless techniques. However, further research is needed to address the challenges associated with their implementation.

### Conclusion

The current state of the art in force and torque estimation for robotic systems presents a diverse array of methodologies, each with its own strengths and limitations. Sensor-based approaches offer high accuracy but come with significant cost and integration challenges. Sensorless methods, while cost-effective and simpler, are heavily dependent on accurate dynamic models and can struggle under varying operational conditions. Kalman filter-based techniques provide robust estimates but require careful calibration and are sensitive to model inaccuracies. Hybrid approaches attempt to combine the strengths of these methods but introduce additional complexity and cost.

The method proposed in this thesis seeks to address these challenges by (insert a detailed description of your method here, including how it builds on the strengths of existing methods while mitigating their weaknesses). By leveraging advancements in (insert key technologies or methodologies), this approach aims to provide a more accurate, efficient, and practical solution for force and torque estimation in robotic systems, particularly those employing harmonic drive transmissions.


# State of the Art

In the field of robotic systems, accurate joint torque estimation is essential for effective force, compliance, and impedance control. Traditional approaches often rely on the use of multiaxis force/torque sensors ($F/T$ sensors) mounted on the robot's structure, such as at the wrist. These sensors enable torque estimation, but they introduce complexities such as additional calculations, potential delays, and susceptibility to model inaccuracies. Moreover, integrating such sensors directly into robotic joints often requires significant modifications to the mechanical structure, which can be impractical.

## Harmonic Drive-Based Torque Estimation

A novel method has been developed to estimate joint torque in robotic joints utilizing harmonic drive transmissions. This method is particularly innovative as it eliminates the need for additional torque sensors or elastic elements, leveraging instead the inherent torsional compliance of the harmonic drive system. The method involves using position measurements from both the motor-side and link-side encoders.

The core of the method relies on the harmonic drive's compliance model, which accounts for the system's nonlinearities, hysteresis, and kinematic errors. The relationship between the angular positions of the wave generator ($\theta_{wg}$) and the flexspline ($\theta_{fs}$) can be described by the kinematic equation:

$$
\theta_{wg} = N\theta_{fs}
$$

where $N$ represents the gear ratio of the harmonic drive. The torque at the wave generator ($\tau_{wg}$) and the flexspline ($\tau_{fs}$) is related by:

$$
\tau_{wg} = \frac{1}{N} \tau_{fs}
$$

The compliance of the flexspline is modeled to capture the nonlinear torsional behavior, expressed as:

$$
\Delta \theta_{fs} = \frac{\arctan(C_{fs}\tau_{fs})}{C_{fs}K_{fs0}}
$$

where $K_{fs0}$ is the initial stiffness of the flexspline, and $C_{fs}$ is a constant that captures the compliance characteristics. These parameters are determined experimentally through calibration.

Finally, the joint torque ($\tau_{fs}$) is estimated using the measured torsional deformation ($\Delta \theta_{fs}$) and the compliance model:

$$
\tau_{fs} = \frac{\tan(\Delta \theta_{fs}C_{fs}K_{fs0})}{C_{fs}}
$$

This method has been validated experimentally, demonstrating its ability to provide accurate torque estimation comparable to traditional $F/T$ sensors, without the need for additional mechanical modifications. This approach simplifies the integration of torque estimation in robotic systems, making it a promising solution for advanced control applications.
